# Project setup

[Project repo link](https://github.com/nicolasugrinovic/size_depth_disambiguation?tab=readme-ov-file)


[Paper link](https://www.iri.upc.edu/people/nugrinovic/depthsize/paper.pdf)

You should run this in GPU runtime.


## Install python packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Dependencies from requirements.txt & packages needed to install pytorch3d
!pip install trimesh open3d pyransac3d fvcore iopath timm

# pytorch3d
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

# Detectron2: source - https://detectron2.readthedocs.io/en/latest/tutorials/install.html
# THIS STEP TAKES LONG (7-8 min)
!python -m pip install pyyaml==5.1
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

# panoptic api
!pip install 'git+https://github.com/cocodataset/panopticapi.git'

## Clone project repo and download external tools

In [ ]:
# clone project repo
!git clone https://github.com/yunijeong5/size_depth_disambiguation.git

# make checkpoints directory to put model
%mkdir /content/size_depth_disambiguation/external/panoptic_deeplab/tools_d2/checkpoints

Before running the cell below, you should download the Panoptic model needed to run the precomputation process. Download the model [here](https://drive.google.com/file/d/17bl_n0SUXVktA0x2507dlwmti7s8FIpe/view?usp=sharing) and save it to your Google Drive.

In [ ]:
# copy model checkpoint from Google Drive. Replace "674 Project" to your folder name.
!cp -r "/content/drive/My Drive/674 Project/panoptic_deeplab_H_48_os16_mg124_poly_200k_bs64_crop_640_640_coco_dsconv.pth" /content/size_depth_disambiguation/external/panoptic_deeplab/tools_d2/checkpoints/panoptic_deeplab_H_48_os16_mg124_poly_200k_bs64_crop_640_640_coco_dsconv.pth

# Download Midas weights
%cd /content/size_depth_disambiguation/
%mkdir -p ./weights
%cd weights
!wget https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt

# Demo


From the Size Depth Disambiguation GitHub:
> To run the demo, you first need to generate (or precompute) data which is then used by the optimization method. You need to generate initial pose/shape estimations and give them the correct format.


One image file (jpg or png), and one obj file and json file (format: `{"joints_3d": [], "translation": []}`) are needed for each person in the photo.

Speicfy the folder with input images and pose/shape estimation with `--input_path` argument.

The image and obj and json files for people in that image should share the same base name.

The optimization process (`run_optim_demo.py`) uses `data_reproj_joints.pkl` generated by the precompute process and files in the input data folder.

Putting all together, the input folder format shold be:
```
input
|-- data_name
    `-- img_name1.jpg
    `-- img_name1_TRANS_person0.obj
    .
    .
    .
    `-- img_name1_TRANS_personN.obj
    `-- img_name1_3djoints_0.json
    .
    .
    .
    
    `-- img_name1_3djoints_N.json
    `-- img_name2.jpg
    .
    .
    .
```

In [ ]:
%cd /content/size_depth_disambiguation/

# precompute the data
!python precompute_estimation_data.py --input_path=./input/coco_demo --output_path=./precomputed_data/coco_demo/ --mode=smpl_reproj --dataset=demo --model_type=dpt_large --config-file ./external/panoptic_deeplab/tools_d2/configs/COCO-PanopticSegmentation/panoptic_deeplab_H_48_os16_mg124_poly_200k_bs64_crop_640_640_coco_dsconv.yaml --opts MODEL.WEIGHTS ./external/panoptic_deeplab/tools_d2/checkpoints/panoptic_deeplab_H_48_os16_mg124_poly_200k_bs64_crop_640_640_coco_dsconv.pth

Issues:
- The reference person is chosen manually: "use normal point equation to translate the plane to the feet sde ref person" ("usa ecuacion punto normal para trasladar el plano a los pie sde ref person", run_optim_demp.py), "ref_person = 1" (run_optim_demp.py).
- The scale of reference person it set to 1, and his translation remains the same as the input.

In [ ]:
# run the optimization
!python run_optim_demo.py --model_type=dpt_large --input_path=./input/coco_demo --output_path=./output/coco_demo --input=input/coco_demo/*.jpg --mode=smpl_reproj --plane_scale=1.0 --n_iters=2000 --w_ordinal_loss=0 --w_reg_size=0.0